# Lecture 2 — Nonlinear systems - Root finding



Today we will cover nonlinear equations and systems of nonlinear equations (single and multivariable)

## Learning Outcomes
By the end of this lecture you will be able to:
- Explain why many engineering problems described by a nonlinear equation (or a system) can be seeing to solving **f(x)=0** or **F(x)=0**.

- Understand **convergence/stop** criteria (function tolerance, step tolerance, iteration cap).

- Distinguish **bracketed** (bisection) vs **open** (Newton) root-finding numerical methods.

- Implement bisection and Newton methods from scratch, as well as use `scipy.optimize` equivalents.

- Understand what can go wrong when employing these methods and how to circumvent such challenges.

- Transition from single to multivariable, system of equations root-finding.

- Be able to use `scipy.optimize.root` to solve systems of nonlinear equations.



## 1) Root finding: exact vs numerical; errors & tolerance
- **Exact** vs **numerical**: most ChemE models need numerical roots. There is no analytical (closed-form) solution to the expression (or set of expressions) that arise. We saw some examples last lecture. A simple example (not ChemE) is:

    $f(x) = ℯ^{-x}+ 0.1 \, \,x \,\, sin(x) = 0$



This nonlinear equation has no simple algebraic solution for $x$. Instead, numerical root-finding methods are used to determine the *approximate solution*. That is, within a tolerance margin.




- We should reduce/simplify the system at hand to **f(x)=0** (or **F(x)=0** in multivariable case, uppercase here means a vector function).
- A numerical method needs tolerances to iterate towards a solution. For example: 

    Function ($|f(x_k)|<\varepsilon_f$), step ($|x_{k+1}-x_k|<\varepsilon_x$), and maximum number of iterations are alternatives.




We stop when one (or more) conditions are met:
- **Function tolerance**: $|f(x_k)| < \varepsilon_f$ — we’re close to a zero/root.
- **Step/interval tolerance**: $|x_{k+1}-x_k| < \varepsilon_x$ or **bisection** width $(b-a) < \varepsilon_x$.
- **Iteration cap**: `max_iter` — prevents infinite loops.

## 2) Interval Halving Methods: Bisection
**Idea.** With a sign-change bracket $[a,b]$ for a continuous $f$, a root exists inside this bracket. This is called the **intermediate value theorem**.


If we halve repeatedly and keep the sign-change subinterval, we will find a solution eventually!




### Bisection Visual Exploration

Let's use an interactive animation to better understand how interval halving methods, such as Bisection, work. 

If you are seeing this in the website version of the lecture, just open this notebook in your google colab session or locally in your machine, by downloading the *.ipynb file.




The example is a nonlinear function defined as $y(x) = e^{-x} - 0.1sin(x)$

In [1]:
import numpy as np, matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import HBox, VBox, Button, IntText, FloatSlider, Checkbox, Output
from IPython.display import display, clear_output
plt.rcParams.update({'figure.figsize': (7.6,5.0), 'axes.labelsize':13, 'axes.titlesize':14})

def f_bisect(x): 
    return np.exp(-x) - 0.1*x*np.sin(x)

def compute_bisection_brackets(f, a, b, nsteps=25, stop_on_exact=True):
    # ensure a <= b
    if a > b:
        a, b = b, a

    aa, bb = float(a), float(b)
    fa, fb = f(aa), f(bb)
    out = []

    # endpoint root?
    if fa == 0 or fb == 0:
        mid = aa if fa == 0 else bb
        out.append((aa, bb, mid))
        return out

    # need a strict sign change to start
    if fa * fb > 0:
        return out  # invalid bracket

    for _ in range(int(nsteps)):
        mid = 0.5 * (aa + bb)
        out.append((aa, bb, mid))
        fm = f(mid)

        if stop_on_exact and fm == 0:
            break

        # keep the subinterval that preserves the sign change
        if fa * fm < 0:
            bb, fb = mid, fm
        else:
            aa, fa = mid, fm

    return out


def bisection_buttons_widget():
    a_slider = FloatSlider(description='a', value=5.0, min=0.5, max=10.0, step=0.01, layout=widgets.Layout(width='360px'), style={'description_width':'15px'})
    b_slider = FloatSlider(description='b', value=8.0, min=0.5, max=10.0, step=0.01, layout=widgets.Layout(width='360px'), style={'description_width':'15px'})
    steps_text = IntText(description='max steps', value=20, layout=widgets.Layout(width='180px'), style={'description_width':'90px'})
    show_hist = Checkbox(value=True, description='Show history', indent=False)

    btn_prev = Button(description='Prev', layout=widgets.Layout(width='120px', height='36px'), disabled=True)
    btn_next = Button(description='Next', layout=widgets.Layout(width='120px', height='36px'))
    btn_reset = Button(description='Reset', layout=widgets.Layout(width='120px', height='36px'))
    iter_text = IntText(description='iter', value=0, layout=widgets.Layout(width='140px'), style={'description_width':'38px'})

    out = Output(layout=widgets.Layout(width='880px', height='520px', border='1px solid #ddd'))
    state={'br':[], 'i':0}

    def recompute():
        state['br']=compute_bisection_brackets(f_bisect, a_slider.value, b_slider.value, int(steps_text.value))
        state['i']=0; iter_text.value=0; btn_prev.disabled=True; btn_next.disabled=(len(state['br'])<=1)

    def render():
        with out:
            clear_output(wait=True)
            x=np.linspace(0.5,10,600); y=f_bisect(x)
            fig,ax=plt.subplots(); ax.axhline(0,color='k',lw=1); ax.plot(x,y,'b-',label='f(x)')
            ax.set_xlim(0.5,11); ax.set_ylim(-2, 2); ax.grid(True)
            br=state['br']
            if not br:
                ax.text(0.5,0.9,'Invalid bracket: f(a)*f(b) ≥ 0', transform=ax.transAxes, color='crimson',
                        bbox=dict(facecolor='mistyrose', edgecolor='crimson')); ax.set_title('Bisection (invalid bracket)'); plt.show(); return
            i=state['i']; ai,bi,mi=br[i]
            if show_hist.value and i>0:
                for (ap,bp,_) in br[:i]:
                    ax.axvspan(ap,bp,color='grey',alpha=0.08); ax.axvline(ap,color='grey',ls='--',lw=1); ax.axvline(bp,color='grey',ls='--',lw=1)
            ax.axvspan(ai,bi,color='gold',alpha=0.25,label='current bracket')
            ax.axvline(ai,color='orange',ls='--',lw=2); ax.axvline(bi,color='orange',ls='--',lw=2)
            ax.plot(mi,f_bisect(mi),'ro',label='midpoint')
            ax.set_title('Bisection: interval halving')
            ax.set_xlabel(f'iter={i}, a={ai:.4f}, b={bi:.4f}, mid={mi:.4f}, |f(mid)|={abs(f_bisect(mi)):.2e}')
            ax.legend(loc='upper left'); plt.show()

    def on_next(_):
        if state['i'] < max(0, len(state['br'])-1):
            state['i']+=1; iter_text.value=state['i']; btn_prev.disabled=False
            if state['i']>=len(state['br'])-1: btn_next.disabled=True
            render()

    def on_prev(_):
        if state['i']>0:
            state['i']-=1; iter_text.value=state['i']; btn_next.disabled=False
            if state['i']<=0: btn_prev.disabled=True
            render()

    def on_reset(_):
        state['i']=0; iter_text.value=0; btn_prev.disabled=True; btn_next.disabled=(len(state['br'])<=1); render()

    def on_param_change(change):
        recompute(); render()

    btn_next.on_click(on_next); btn_prev.on_click(on_prev); btn_reset.on_click(on_reset)
    a_slider.observe(on_param_change, names='value'); b_slider.observe(on_param_change, names='value')
    steps_text.observe(on_param_change, names='value'); show_hist.observe(on_param_change, names='value')
    recompute(); render()
    display(VBox([HBox([a_slider,b_slider,steps_text,show_hist]), HBox([btn_prev,btn_next,btn_reset,iter_text]), out]))

bisection_buttons_widget()

How does this in code looks like? Let's see one possible implementation

In [2]:
import numpy as np


def bisection(f, a, b, ftol=1e-10, xtol=1e-12, max_iter=200):

    """
    Bisection's Method.

    Inputs
    ------
    f: function
        The function we want to find the root.
    
    a: float
        Lower bound of the initial interval.
    b: float
        Upper bound of the initial interval.
    ftol: float
        Tolerance for absolute function value (default: 1e-10)
    xtol: float
        Tolerance for the step length (default: 1e-12)
    max_iter: int
        Max number of iterations (default: 200)

    Returns
    -------
    m: float
        Approximate root of f(x)
    iter: int
        Number of iterations.
    
    fm: float
        Value of f(x) at the solution.    
    """
    fa = f(a)
    fb = f(b)

    if fa * fb > 0:
        raise ValueError("f(a)*f(b)>0, hence we don't have a sign change. Exit.")


    for k in range(max_iter):
        m = 0.5 * (a + b)

        fm = f(m)

        if np.abs(fm) < ftol or np.abs((b-a)) < xtol:
            return m, k+1, fm

        if fa * fm < 0:
            b, fb = m, fm
        else:
            a, fa = m, fm
    
    m =  0.5 * (a + b)

    fm = f(m)


    return m, max_iter, fm

In [3]:
def f_bisect(x): 
    return np.exp(-x) - 0.1*x*np.sin(x)

a = 5.00
b = 8.00
solution = bisection(f_bisect, a, b)

print(solution)

(6.286147252423689, 32, np.float64(-2.1169849293706244e-11))


A few notes regarding the bisection method:



- We will always find a solution, given that the intermediate value theorem is true when choosing the interval $[a,b]$
- Bisection can be a bit slow to converge. It has linear convergence.

This leaves an open question of "can we have faster methods?" Let's see it below

## 3) Open Methods: Newton's Method

In the open methods, we don't need an interval $[a,b]$ to try to find a solution. Instead, we need only an initial estimate $x_0$. This might look an absolute improvement at a first glance, since these methods tend to converge faster in general, but there are a few caveats. First, let's take a look at the iterative equation for Newton's method:

**Newton's method formula:** $x_{k+1}=x_k-\dfrac{f(x_k)}{f'(x_k)}$ 

How do we even got this formula in the first place? 

### Newton’s Method derivation via Taylor Series Expansion (TSE)

#### Reminder: What is a Taylor Series?
If a function $f(x)$ is sufficiently smooth (that is, we can take derivatives), we are able to approximate it near a point, say, $x=a$ by its derivatives at $a$:

$f(x) = f(a) + f'(a)(x-a) + \frac{f''(a)}{2!}(x-a)^2 + \frac{f^{(3)}(a)}{3!}(x-a)^3 + \cdots$
- The first term is the value of $f$ at $a$.

- The second term adds the local linear (slope) correction.

- Higher-order terms capture curvature and what we call "high-order" effects.

- When $x$ is close to $a$, the higher-order terms are small.



#### Apply Taylor Expansion to Root Finding
Remember our goal: solve $f(x)=0$ ! (exclamation mark, not factorial :) ) 

Let $x_k$ be the current guess. Expand $f$ with a $1^{st}$ order TSE expansion around $x_k$:
$f(x) \approx f(x_k) + f'(x_k)(x - x_k)$

We know $x_k$ and hence, can calculate $f(x_k)$ and $f'(x_k)$

#### Impose the Root Condition
At the next iterate $x_{k+1}$ we want $f(x_{k+1})=0$. After all, we want it to converge, correct? 

Let's impose this condition then. $f(x_{k+1})=0$:

$(x_{k+1})=0 \approx f(x_k) + f'(x_k)(x_{k+1} - x_k)$


Solve for $x_{k+1}$ to get the Newton update (also called newton step):


$x_{k+1} = x_k - \frac{f(x_k)}{f'(x_k)}$

This is also written sometimes as 

$x_{k+1} = x_k + d_k$ 

$d_k = - \frac{f(x_k)}{f'(x_k)}$


#### Why is Newton’s method fast?

Let the root be $r$ and define the error $e_k = x_k - r$

Start from the **full Taylor series of $f$ about $r$** (with $x_k = r + e_k$):


$$
f(x_k) = f(r+e_k)
      = f(r) + f'(r)e_k + \frac{f''(r)}{2!}e_k^2 + \frac{f^{(3)}(r)}{3!}e_k^3 + \cdots
$$

Since $f(r)=0$, this simplifies to


$$
f(x_k) = f'(r)e_k + \frac{f''(r)}{2}e_k^2 + \frac{f^{(3)}(r)}{6}e_k^3 + \cdots
$$


If $x_k$ is close to $r$ (so $|e_k|$ is small), higher-order terms are negligible, giving


$$
f(x_k) \approx f'(r)e_k + \tfrac{1}{2} f''(r)e_k^2
$$

Newton’s update is

$$
x_{k+1} = x_k - \frac{f(x_k)}{f'(x_k)}
$$

Hence the next error is

$$
e_{k+1} = x_{k+1} - r = e_k - \frac{f(x_k)}{f'(x_k)}
$$

Substitute the Taylor approximation for $f(x_k)$ and (near $r$) replace $f'(x_k)$ by $f'(r)$:

$$f'(x_k) = f'(r + e_k)
= f'(r) + f''(r)e_k + \tfrac12 f^{(3)}(r)e_k^2 + \cdots \approx f'(r)
$$

$$
e_{k+1} \approx e_k - \frac{f'(r)e_k + \tfrac{1}{2}f''(r)e_k^2}{f'(r)}
          = e_k - (e_k - \frac{f''(r)}{2f'(r)} e_k^2)
          = \frac{f''(r)}{2f'(r)} e_k^2
$$

Thus,

$$
e_{k+1} \propto e_k^2
$$

The new error is proportional to the **square** of the old error (quadratic convergence). 
 
If $e_k \sim 10^{-2}$, then $e_{k+1} \sim 10^{-4}$.  

By contrast, bisection has only linear convergence: $e_{k+1} \approx \tfrac{1}{2}e_k$.

### Newton's Method Visual Exploration

This works in your jupyter notebook sesssion. Not in the website. Let's explore Newton's method! 

Again, you should not worry about the code that generated the anmiations below. It's just the use of [jupyter widgets](https://ipywidgets.readthedocs.io/en/latest/) (and a bit of prompting using ChatGPT to get the widget working well) so we could generate an interactive visualization of how Newton's method works :)

In [4]:
import numpy as np, matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import HBox, VBox, Button, IntText, FloatSlider, Output
from IPython.display import display, clear_output

# Clean, well-behaved cubic: roots at 1, 2, 4
def f(x):
    return x**3 - 7*x**2 + 14*x - 8

def fprime(x):
    return 3*x**2 - 14*x + 14

def newton_sequence(f, fprime, x0, max_steps=25, ftol=1e-12, xtol=1e-12):
    xs = [float(x0)]
    for _ in range(int(max_steps)):
        x = xs[-1]
        fx = f(x)
        dfx = fprime(x)
        if abs(dfx) < 1e-14:   # safeguard against tiny slope
            break
        xnew = x - fx / dfx
        xs.append(xnew)
        # test convergence at the new iterate
        if abs(f(xnew)) < ftol or abs(xnew - x) < xtol:
            break
    return xs

def newton_buttons_widget():
    plt.rcParams.update({'figure.figsize': (7.6,5.0), 'axes.labelsize':13, 'axes.titlesize':14})

    x0_slider = FloatSlider(description='x0', value=1.5, min=0.0, max=5.0, step=0.01,
                            layout=widgets.Layout(width='360px'), style={'description_width':'20px'})
    steps_text = IntText(description='max steps', value=10,
                         layout=widgets.Layout(width='180px'), style={'description_width':'90px'})
    btn_prev = Button(description='Prev', layout=widgets.Layout(width='120px', height='36px'), disabled=True)
    btn_next = Button(description='Next', layout=widgets.Layout(width='120px', height='36px'))
    btn_reset = Button(description='Reset', layout=widgets.Layout(width='120px', height='36px'))
    iter_text = IntText(description='iter', value=0,
                        layout=widgets.Layout(width='140px'), style={'description_width':'38px'})
    out = Output(layout=widgets.Layout(width='880px', height='520px', border='1px solid #ddd'))

    state = {'xs': [], 'i': 0}

    def recompute():
        state['xs'] = newton_sequence(f, fprime, x0_slider.value, int(steps_text.value))
        state['i'] = 0
        iter_text.value = 0
        btn_prev.disabled = True
        btn_next.disabled = (len(state['xs']) <= 1)

    def render():
        with out:
            clear_output(wait=True)
            X = np.linspace(0, 5, 600); Y = f(X)

            fig, ax = plt.subplots()
            ax.axhline(0, color='k', lw=1)
            ax.plot(X, Y, 'b-', label='f(x)')
            ax.set_xlim(0, 5)

            # dynamic y-limits with margin
            yspan = Y.max() - Y.min()
            pad = 0.08*yspan if yspan > 0 else 1.0
            ax.set_ylim(Y.min()-pad, Y.max()+pad)
            ax.grid(True)

            # mark true roots for reference
            for r in (1.0, 2.0, 4.0):
                ax.axvline(r, color='orange', ls='--', lw=1.2, alpha=0.85)
                ax.text(r, 0.02*(Y.max()-Y.min())+0, f"x={r:g}", color='orange', ha='center')

            xs = state['xs']

            i = state['i']; xn = xs[i]; yn = f(xn)
            ax.plot(xn, yn, 'ro', label='current iterate')

            if i < len(xs) - 1:
                slope = fprime(xn)
                tangent = yn + slope*(X - xn)
                ax.plot(X, tangent, 'g--', lw=1.4, label='tangent at $x_n$')
                ax.axvline(xs[i+1], color='purple', ls='--', lw=1.5)
                ax.plot(xs[i+1], 0, 'mo', label='$x_{n+1}$')

            ax.set_title("Newton's Method on $x^3 - 7x^2 + 14x - 8$")
            yn_abs = np.abs(yn)
            ax.set_xlabel(f'iter={i}, x={xn:.6f}, |f(x)|={yn_abs:.2e}')
            ax.legend(loc='upper right')
            plt.show()

    def on_next(_):
        if state['i'] < max(0, len(state['xs']) - 1):
            state['i'] += 1
            iter_text.value = state['i']
            btn_prev.disabled = False
            if state['i'] >= len(state['xs']) - 1:
                btn_next.disabled = True
            render()

    def on_prev(_):
        if state['i'] > 0:
            state['i'] -= 1
            iter_text.value = state['i']
            btn_next.disabled = False
            if state['i'] <= 0:
                btn_prev.disabled = True
            render()

    def on_reset(_):
        state['i'] = 0
        iter_text.value = 0
        btn_prev.disabled = True
        btn_next.disabled = (len(state['xs']) <= 1)
        render()

    def on_param_change(change):
        recompute(); render()

    btn_next.on_click(on_next)
    btn_prev.on_click(on_prev)
    btn_reset.on_click(on_reset)
    x0_slider.observe(on_param_change, names='value')
    steps_text.observe(on_param_change, names='value')

    recompute(); render()
    display(VBox([HBox([x0_slider, steps_text]), HBox([btn_prev, btn_next, btn_reset, iter_text]), out]))

newton_buttons_widget()

An example in which things can go wrong:

In [ ]:
import numpy as np, matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import HBox, VBox, Button, IntText, FloatSlider, Output
from IPython.display import display, clear_output

# tanh example (can diverge for some x0, e.g., 1.1)
def f(x):
    return np.tanh(x)

def fprime(x):
    t = np.tanh(x)
    return 1.0 - t*t  # sech^2(x)

def newton_sequence(f, fprime, x0, max_steps=25, ftol=1e-12, xtol=1e-12):
    xs = [float(x0)]
    for _ in range(int(max_steps)):
        x = xs[-1]
        fx = f(x)
        dfx = fprime(x)
        if abs(dfx) < 1e-14:   # safeguard against tiny slope
            break
        xnew = x - fx / dfx
        xs.append(xnew)
        # test convergence at the new iterate
        if abs(f(xnew)) < ftol or abs(xnew - x) < xtol:
            break
    return xs

def newton_buttons_widget():
    plt.rcParams.update({'figure.figsize': (7.6,5.0), 'axes.labelsize':13, 'axes.titlesize':14})

    x0_slider = FloatSlider(description='x0', value=1.1, min=-4.0, max=4.0, step=0.01,
                            layout=widgets.Layout(width='360px'), style={'description_width':'20px'})
    steps_text = IntText(description='max steps', value=10,
                         layout=widgets.Layout(width='180px'), style={'description_width':'90px'})
    btn_prev = Button(description='Prev', layout=widgets.Layout(width='120px', height='36px'), disabled=True)
    btn_next = Button(description='Next', layout=widgets.Layout(width='120px', height='36px'))
    btn_reset = Button(description='Reset', layout=widgets.Layout(width='120px', height='36px'))
    iter_text = IntText(description='iter', value=0,
                        layout=widgets.Layout(width='140px'), style={'description_width':'38px'})
    out = Output(layout=widgets.Layout(width='880px', height='520px', border='1px solid #ddd'))

    state = {'xs': [], 'i': 0}

    def recompute():
        state['xs'] = newton_sequence(f, fprime, x0_slider.value, int(steps_text.value))
        state['i'] = 0
        iter_text.value = 0
        btn_prev.disabled = True
        btn_next.disabled = (len(state['xs']) <= 1)

    def render():
        with out:
            clear_output(wait=True)
            X = np.linspace(-4, 4, 600); Y = f(X)

            fig, ax = plt.subplots()
            ax.axhline(0, color='k', lw=1)
            ax.plot(X, Y, 'b-', label='f(x) = tanh(x)')
            ax.set_xlim(-4, 4)

            # dynamic y-limits with margin (same style as cubic demo)
            yspan = Y.max() - Y.min()
            pad = 0.08*yspan if yspan > 0 else 1.0
            ax.set_ylim(Y.min()-pad, Y.max()+pad)
            ax.grid(True)

            xs = state['xs']
            i = state['i']; xn = xs[i]; yn = f(xn)
            ax.plot(xn, yn, 'ro', label='current iterate')

            if i < len(xs) - 1:
                slope = fprime(xn)
                tangent = yn + slope*(X - xn)
                ax.plot(X, tangent, 'g--', lw=1.4, label='tangent at $x_n$')
                ax.axvline(xs[i+1], color='purple', ls='--', lw=1.5)
                ax.plot(xs[i+1], 0, 'mo', label='$x_{n+1}$')

            ax.set_title("Newton's Method on $\\tanh(x)$")
            ax.set_xlabel(f'iter={i}, x={xn:.6f}, |f(x)|={abs(yn):.2e}')
            ax.legend(loc='upper right')
            plt.show()

    def on_next(_):
        if state['i'] < max(0, len(state['xs']) - 1):
            state['i'] += 1
            iter_text.value = state['i']
            btn_prev.disabled = False
            if state['i'] >= len(state['xs']) - 1:
                btn_next.disabled = True
            render()

    def on_prev(_):
        if state['i'] > 0:
            state['i'] -= 1
            iter_text.value = state['i']
            btn_next.disabled = False
            if state['i'] <= 0:
                btn_prev.disabled = True
            render()

    def on_reset(_):
        state['i'] = 0
        iter_text.value = 0
        btn_prev.disabled = True
        btn_next.disabled = (len(state['xs']) <= 1)
        render()

    def on_param_change(change):
        recompute(); render()

    btn_next.on_click(on_next)
    btn_prev.on_click(on_prev)
    btn_reset.on_click(on_reset)
    x0_slider.observe(on_param_change, names='value')
    steps_text.observe(on_param_change, names='value')

    recompute(); render()
    display(VBox([HBox([x0_slider, steps_text]), HBox([btn_prev, btn_next, btn_reset, iter_text]), out]))

newton_buttons_widget()

It quickly diverges! If we change the initial step slightly, say, $x_0=0.9$, it converges. Try it yourself in the interface above.

###   Implementation of Newton’s Method (scalar case)

In [6]:
import numpy as np


def newton(f, fprime, x0, tol=1e-8, max_iter = 50):
    """
    Newton's method.


    Inputs
    ------

    f: function
        The function we want to determine the root.

    fprime: function
        The derivative of the function f
    x0: float
        Initial estimate (guess).
    tol: float
        Tolerance.
    max_iter:
        Maximum number of iterations...


    Returns
    -------

    x: float
        Approximate root of f(x)
    iter: int
        Number of iterations
    sol: float
        Value of f(x) at the solution.
    
    
    """

    x = x0

    if np.abs(f(x)) < tol:
        return x
    
    # Main for loop:

    for k in range(max_iter):

        x_new = x - f(x)/fprime(x)  # Newton's update formula

        if np.abs(f(x_new)) < tol: # Function value small enough. stop.
            return x_new, k+1, np.abs(f(x_new))
        if np.abs(x_new - x) < 1e-14: # Step size small enough. Stop.
            ## TODO: Maybe we should add 1e-14 as a keyword argument instead of hard coding it.
            return x_new, k+1, np.abs(f(x_new))
        
        x = x_new

    
    fx = f(x)
    x = x_new

    raise ValueError(f"Newton's method did not converge after {k+1} iterations;", f"x={x:.6f}, f(x)={fx:.6f}")



Let's try a $3^{rd}$ order degree polynomial, defined as $f(x) = x^3 - 7x^2 + 14x - 8$ and with derivative defined as  $f'(x)=3x^2 - 14x + 14$

In [7]:
def f(x):
    return x**3 - 7*x**2 + 14*x - 8

def fprime(x):
    return 3*x**2 - 14*x + 14

Solving using our function `newton`.

In [8]:
x, k, sol = newton(f, fprime, 0.7, tol=1e-8, max_iter=50)

print('optimal solution is x=', x)
print('number of iterations =', k)
print('f(x)=', sol)

optimal solution is x= 0.9999999967198463
number of iterations = 4
f(x)= 9.840460890586655e-09


## 4) Using SciPy: `bisect`, `newton`

Scipy already contains implementations of bisection and Newton's method. We will go over the use of these now. But their functionality is no different from what we just saw in class.

- `optimize.bisect(f, a, b)`: robust with a bracket.
- `optimize.newton(f, x0, fprime=...)`: Newton with derivative


Let's try another example, and compare our implementation with scipy's. Let's take a look at $f(x) = x-cos(x)$

In [9]:
from scipy import optimize
import numpy as np

# Defining the function and its derivative (for Newton's method)
def f(x):      return x - np.cos(x)
def fprime(x): return 1 + np.sin(x)

# Let's solve all of these
print('SciPy bisection, x =', optimize.bisect(f, 0.0, 1.0, xtol=1e-12, rtol=1e-12, maxiter=200))
print('SciPy newton, x =', optimize.newton(f, x0=0.7, fprime=fprime, tol=1e-12, maxiter=50))
print('Our implementation of bisection, x =', bisection(f, 0.0, 1.0)[0])
print('Our implementation of Newton\'s method, x =', newton(f, fprime, x0=0.7)[0])

SciPy bisection, x = 0.7390851332156672
SciPy newton, x = 0.7390851332151607
Our implementation of bisection, x = 0.7390851331874728
Our implementation of Newton's method, x = 0.7390851332151608


Now, let's try the example that was used in the demo above for Newton's method, that is $f(x) = x^3 + 7x^2 + 14x -8$.

The derivative for this is $f'(x) = 3x^2 - 14x + 14$. 

This equation has three roots as we saw in the animation above (1, 2 and 4)

In [10]:
def f(x):
    return x**3 - 7*x**2 + 14*x - 8

def fprime(x):
    return 3*x**2 - 14*x + 14


print('SciPy bisection, x = :', optimize.bisect(f, 0.0, 1.46, xtol=1e-12, rtol=1e-12, maxiter=200))
print('SciPy newton, x = :', optimize.newton(f, x0=1.45, fprime=fprime, tol=1e-12, maxiter=50))
print('Our implementation of bisection, x =', bisection(f, 0.0, 1.5)[0])
print('Our implementation of Newton\'s method, x =', newton(f, fprime, x0=1.45)[0])

SciPy bisection, x = : 1.0000000000000548
SciPy newton, x = : 0.9999999999999999
Our implementation of bisection, x = 1.0000000000291038
Our implementation of Newton's method, x = 0.9999999999986046


In [11]:
print('SciPy bisection, x = :', optimize.bisect(f, 1.5, 2.5, xtol=1e-12, rtol=1e-12, maxiter=200))
print('SciPy newton, x = :', optimize.newton(f, x0=1.5, fprime=fprime, tol=1e-12, maxiter=50))
print('Our implementation of bisection, x =', bisection(f, 1.5, 2.5)[0])
print('Our implementation of Newton\'s method, x =', newton(f, fprime, x0=1.5)[0])

SciPy bisection, x = : 2.0
SciPy newton, x = : 4.0
Our implementation of bisection, x = 2.0
Our implementation of Newton's method, x = 4.0


In [12]:
from scipy import optimize
import numpy as np

def f(x):
    return np.tanh(x)

def fprime(x):
    t = np.tanh(x)
    return 1.0 - t*t  # sech^2(x)

print('SciPy newton, x = :', optimize.newton(f, x0=1.08, fprime=fprime, tol=1e-12, maxiter=50))
print('Our implementation of Newton\'s method, x =', newton(f, fprime, x0=1.01)[0])

SciPy newton, x = : 0.0
Our implementation of Newton's method, x = -5.025528739899601e-12


#### Extension to systems: Newton’s method in higher dimensions

It is not uncommon that instead of needing to solve a single nonlinear function, we have a system of nonlinear equations. These arise all the time in chemical reaction engineering, process design and control, process dynamics, and so on. These all share in common the particular feature that now, the unknown is a **vector** $x_k \in \mathbb{R}^n$.  
We put $n$ nonlinear equations into what we call a vector function $F:\mathbb{R}^n \to \mathbb{R}^n$:

$$
F(x) =
\begin{bmatrix}
f_1(x_1,\dots,x_n) \\
\vdots \\
f_n(x_1,\dots,x_n)
\end{bmatrix},
\qquad 
$$

And we want $F(x)=0$. Nothing has changed in our goal (root finding), we now just have more equations to deal with simultaneously. This is actually **a system of nonlinear equations**, and hence, we have to use the appropriate mathematical concepts to deal with this increased dimensionality.

#### Jacobian
The **Jacobian** $J(x)$ is the matrix of all first derivatives:

$$
J(x) = \left[\frac{\partial f_i}{\partial x_j}(x)\right]_{i,j=1}^n
$$

Expanding the notation above you can see the full matrix as:

$$
J(x) = 
\begin{bmatrix}
\dfrac{\partial f_1}{\partial x_1}(x) & \dfrac{\partial f_1}{\partial x_2}(x) & \cdots & \dfrac{\partial f_1}{\partial x_n}(x) \\
\dfrac{\partial f_2}{\partial x_1}(x) & \dfrac{\partial f_2}{\partial x_2}(x) & \cdots & \dfrac{\partial f_2}{\partial x_n}(x) \\
\vdots & \vdots & \ddots & \vdots \\
\dfrac{\partial f_n}{\partial x_1}(x) & \dfrac{\partial f_n}{\partial x_2}(x) & \cdots & \dfrac{\partial f_n}{\partial x_n}(x)
\end{bmatrix}
$$

And here we show that the Newton's method is conceptually the same for the multivariate case. If we expand $F$ employing Taylor around $x_k$:

$$
F(x) \approx F(x_k) + J(x_k)(x - x_k)
$$

And impose the root condition at $x_{k+1}$, that is, we want $F(x_{k+1}) = 0$, we have:

$$
0 \approx F(x_k) + J(x_k)(x_{k+1} - x_k)
$$

Lastly, if we solve for $x_{k+1}$:

$$
J(x_k)(x_{k+1} - x_k) = -F(x_k),
$$
$$
x_{k+1} = x_k + d_k, \quad d_k \text{ solves } J(x_k)d_k = -F(x_k)
$$

Equivalently, we can write this equation as 

$$
x_{k+1} = x_k - J(x_k)^{-1} F(x_k)
$$

But we avoid this in practice since evaluating an inverse is expensive from a computational standpoint. Instead, we solve the linear system posed by $J(x_k)d_k = -F(x_k)$



#### Side-by-side: scalar vs. vector versions of Newton's method

To further stress the fact that these are the same algorithm, just adapted for higher dimensions, please take a look at the table below for a quick comparison:

| Concept | Scalar (1D) | Vector (nD) |
|---|---|---|
| Unknown | $x_k \in \mathbb{R}$ | $x_k \in \mathbb{R}^n$ |
| Function | $f:\mathbb{R}\to\mathbb{R}$ | $F:\mathbb{R}^n\to\mathbb{R}^n$ |
| Expansion | $f(x)\approx f(x_k)+f'(x_k)(x-x_k)$ | $F(x)\approx F(x_k)+J(x_k)(x-x_k)$ |
| Root condition | $0\approx f(x_k)+f'(x_k)(x_{k+1}-x_k)$ | $0\approx F(x_k)+J(x_k)(x_{k+1}-x_k)$ |
| Newton step | $d_k=-\tfrac{f(x_k)}{f'(x_k)}$ | $J(x_k)d_k=-F(x_k)$ |
| Update | $x_{k+1}=x_k+d_k$ | $x_{k+1}=x_k+d_k$ |



#### Example: $2\times 2$ generic system
Suppose

$$
F(x) =
\begin{bmatrix}
f_1(x_1,x_2) \\
f_2(x_1,x_2)
\end{bmatrix},
\quad
J(x) =
\begin{bmatrix}
\frac{\partial f_1}{\partial x_1} & \frac{\partial f_1}{\partial x_2}\\
\frac{\partial f_2}{\partial x_1} & \frac{\partial f_2}{\partial x_2}
\end{bmatrix}
$$

The Newton step solves


$$
\begin{bmatrix}
\frac{\partial f_1}{\partial x_1} & \frac{\partial f_1}{\partial x_2}\\
\frac{\partial f_2}{\partial x_1} & \frac{\partial f_2}{\partial x_2}
\end{bmatrix}_{x_k}
\begin{bmatrix}
d_{k,1}\\
d_{k,2}
\end{bmatrix}
= -
\begin{bmatrix}
f_1(x_k)\\
f_2(x_k)
\end{bmatrix}
$$

Then $x_{k+1} = x_k + d_k$.

## Can we code this in Python? (We continue from here on Sep 3rd. Your first homework does not require the code development below.)

Yup! It's pretty much the same idea:

In [13]:
import numpy as np


def newton_nd(F, J, x0, tol=1e-8, max_iter = 50):
    """
    Newton's method (multivariate).


    Inputs
    ------

    f: function
        The vector-valued function we want to determine the root.

    J: function
        The Jacobian of the function F
    x0: float
        Initial estimate (guess).
    tol: float
        Tolerance.
    max_iter:
        Maximum number of iterations...


    Returns
    -------

    x: float
        Approximate root of f(x)
    iter: int
        Number of iterations
    sol: float
        Value of f(x) at the solution.
    
    
    """

    x = x0

    if np.linalg.norm(F(x), ord =2)  < tol:
        return x, 0, F(x) 
    # Main for loop:

    for k in range(max_iter):

       

        x_new = x - np.linalg.inv(J(x)) @ F(x)

        if np.linalg.norm(F(x_new)) < tol: # Function value small enough. stop.
            return x_new, k+1, F(x_new)
        if np.linalg.norm(x_new - x) < 1e-14: # Step size small enough. Stop.
            ## TODO: Maybe we should add 1e-14 as a keyword argument instead of hard coding it.
            return x_new, k+1, F(x_new)
        
        x = x_new

    
    Fx = F(x)
    x = x_new

    raise ValueError(f"Newton's method did not converge after {k+1} iterations;", f"x={x:.6f}, f(x)={F:.6f}")



Let's try this for a system defined as 


$$
\begin{align*}
2 x_1^2 + x_2^2 = 6 \\
x_1 + 2 x_2 = 3.5
\end{align*}
$$

In [14]:
# Define F and J for the system
def F(x):
    x1, x2 = x
    return np.array([
        2.0*x1**2 + x2**2 - 6.0,
        x1 + 2.0*x2 - 3.5
    ])

def J(x):
    x1, x2 = x
    return np.array([
        [4.0*x1, 2.0*x2],
        [1.0,    2.0   ]
    ])

# Two different initial guesses
x0_1 = np.array([1.0, 1.0])      
x0_2 = np.array([-1.0, 2.2])     

root1, it1, res1 = newton_nd(F, J, x0_1, tol=1e-10, max_iter=50)
root2, it2, res2 = newton_nd(F, J, x0_2, tol=1e-10, max_iter=50)

print("From x0 =", x0_1, "root =", root1, "in", it1, "iters; residual =", res1)
print("From x0 =", x0_2, "root =", root2, "in", it2, "iters; residual =", res2)

From x0 = [1. 1.] root = [1.5958645  0.95206775] in 5 iters; residual = [8.8817842e-16 0.0000000e+00]
From x0 = [-1.   2.2] root = [-0.81808672  2.15904336] in 4 iters; residual = [8.8817842e-16 0.0000000e+00]


# What about using scipy?

You can use `scipy.optimize.root` to solve the exact same problem (or any root finding problem). Let's take a look.

In [15]:
import scipy

help(scipy.optimize.root)

Help on function root in module scipy.optimize._root:

root(
    fun,
    x0,
    args=(),
    method='hybr',
    jac=None,
    tol=None,
    callback=None,
    options=None
)
    Find a root of a vector function.

    Parameters
    ----------
    fun : callable
        A vector function to find a root of.

        Suppose the callable has signature ``f0(x, *my_args, **my_kwargs)``, where
        ``my_args`` and ``my_kwargs`` are required positional and keyword arguments.
        Rather than passing ``f0`` as the callable, wrap it to accept
        only ``x``; e.g., pass ``fun=lambda x: f0(x, *my_args, **my_kwargs)`` as the
        callable, where ``my_args`` (tuple) and ``my_kwargs`` (dict) have been
        gathered before invoking this function.
    x0 : ndarray
        Initial guess.
    args : tuple, optional
        Extra arguments passed to the objective function and its Jacobian.
    method : str, optional
        Type of solver. Should be one of

        - 'hybr'             

You can also do `?scipy.optimize.root`. Same idea.

In [16]:
?scipy.optimize.root

Signature:
scipy.optimize.root(
    fun,
    x0,
    args=(),
    method='hybr',
    jac=None,
    tol=None,
    callback=None,
    options=None,
)
Docstring:
Find a root of a vector function.

Parameters
----------
fun : callable
    A vector function to find a root of.

    Suppose the callable has signature ``f0(x, *my_args, **my_kwargs)``, where
    ``my_args`` and ``my_kwargs`` are required positional and keyword arguments.
    Rather than passing ``f0`` as the callable, wrap it to accept
    only ``x``; e.g., pass ``fun=lambda x: f0(x, *my_args, **my_kwargs)`` as the
    callable, where ``my_args`` (tuple) and ``my_kwargs`` (dict) have been
    gathered before invoking this function.
x0 : ndarray
    Initial guess.
args : tuple, optional
    Extra arguments passed to the objective function and its Jacobian.
method : str, optional
    Type of solver. Should be one of

    - 'hybr'             :ref:`(see here) <optimize.root-hybr>`
    - 'lm'               :ref:`(see here) <optimiz

In [17]:
x0_1 = np.array([1.0, 1.0])      
x0_2 = np.array([-1, -1])     



x_soln_1 = scipy.optimize.root(fun=F, x0=x0_1)
print(x_soln_1.x) 

x_soln_2 = scipy.optimize.root(fun=F, x0=x0_2)
print(x_soln_2.x) 


[1.5958645  0.95206775]
[-0.81808672  2.15904336]


There is a small detail here. We did not supply a Jacobian to `scipy.optimize.root`. Why do you think that happened? The method needs derivatives/jacobians, but we did not supply one. Why it still worked?

But if we do supply a Jacobian, what happens?

In [18]:
x_soln_1_jac = scipy.optimize.root(fun=F, x0=x0_1, jac=J)

print(x_soln_1.x) 
print(x_soln_1_jac.x) 

[1.5958645  0.95206775]
[1.5958645  0.95206775]


The solutions are the same. But what about number of iterations? Let us print both (without and with) exact Jacobian:

In [19]:
print(x_soln_1) # Without providing the Jacobian function we created.

 message: The solution converged.
 success: True
  status: 1
     fun: [ 0.000e+00  0.000e+00]
       x: [ 1.596e+00  9.521e-01]
  method: hybr
    nfev: 12
    fjac: [[-9.876e-01 -1.572e-01]
           [ 1.572e-01 -9.876e-01]]
       r: [-6.360e+00 -1.992e+00 -1.708e+00]
     qtf: [ 1.086e-10 -1.728e-11]


In [20]:
print(x_soln_1_jac) # Providing the Jacobian function we created.

 message: The solution converged.
 success: True
  status: 1
     fun: [ 0.000e+00  0.000e+00]
       x: [ 1.596e+00  9.521e-01]
  method: hybr
    nfev: 10
    njev: 1
    fjac: [[-9.876e-01 -1.572e-01]
           [ 1.572e-01 -9.876e-01]]
       r: [-6.360e+00 -1.992e+00 -1.708e+00]
     qtf: [ 1.086e-10 -1.728e-11]


⚠️ Important ⚠️

It took `scipy.optimize.root` fewer iterations (nfev argument printed in the output, 12 vs 10). Why is that?

⚠️ Important ⚠️

Let's take a look into *derivative approximations*

## Wrap-up & references
- **Bisection** is reliable in a bracket, but slow. We don't always have a bracket though.
- **Newton** is fast with good starting points, but can easily diverge or even find different solutions from what we anticipated (it is based on derivative information).
- For systems of nonlinear equations, provide **Jacobians**; `scipy.optimize.root` is robust in practice.


**Docs and References used**  
- SciPy Optimize:  https://docs.scipy.org/doc/scipy/reference/optimize.html  
- PyCSE (Kitchin): https://kitchingroup.cheme.cmu.edu/pycse/  
- Harishankar Manikantan, *Solutions of Nonlinear Equations (Chapter 6)*, ECH60 course notes (GitHub: hmanikantan/ECH60): https://github.com/hmanikantan/ECH60


